In [69]:
import arcpy
import psycopg2
import os
import random

#### Get Current Directory and Create Geodatabase

In [8]:
directory = os.getcwd()
arcpy.env.workspace = directory
gdb = "Lab3"

# Check if the geodatabase exists
if not arcpy.Exists(gdb + ".gdb"):
    # Create the geodatabase
    arcpy.CreateFileGDB_management(arcpy.env.workspace, gdb)
    print("Geodatabase created.")
else:
    print("Geodatabase already exists.")
    
db = os.path.join(directory + "\Lab3.gdb")

Geodatabase already exists.


### Connect to Cloud Geodatabase

In [58]:
# Connect to the database
connection = psycopg2.connect(
    dbname="gis5572",
    user=user,
    password=password,
    host=host,
    port="5432"
)

#create connection to cloud database
out_folder_path = arcpy.env.workspace
out_name = "Lab3.sde"
database_platform = "POSTGRESQL"
instance = host
account_authentication = "DATABASE_AUTH"
username = user
password = password
save_user_pass = "SAVE_USERNAME"
database = "gis5572"
#actual database connection
arcpy.management.CreateDatabaseConnection(out_folder_path, out_name, database_platform, instance, account_authentication, username, password, save_user_pass, database)

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\5572-Lab3\\Lab3.sde'>

### Get Elevation Data

In [59]:
# Create cursor to go through data
cursor = connection.cursor()
# Get point data using query
query = "SELECT objectid, ST_AsText(shape), pointid, grid_code FROM elevation_point"
cursor.execute(query)
rows = cursor.fetchall()

# Create  feature class
elevation_fc = os.path.join(db, "elevation_points")
spatial_ref = arcpy.SpatialReference(26915)
if arcpy.Exists(elevation_fc):
    arcpy.Delete_management(elevation_fc)
arcpy.CreateFeatureclass_management(db, "elevation_points", "POINT", spatial_reference=spatial_ref)

# Add a field to store additional attributes if needed
arcpy.AddField_management(elevation_fc, "pointid", "LONG")
arcpy.AddField_management(elevation_fc, "grid_code", "LONG")

# Create an insert cursor
insert_cursor = arcpy.da.InsertCursor(elevation_fc, ["Shape", "pointid", "grid_code"])  

# Iterate through the rows and insert points into the feature class
for row in rows:
    point_wkt = row[1]
    pointid = row[2]
    grid_code = row[3]
    # Extract x, y coordinates from WKT
    point_geom = arcpy.FromWKT(point_wkt, arcpy.SpatialReference(26915))
    # Insert the point into the feature class
    insert_cursor.insertRow([point_geom, pointid, grid_code])

# Close cursors and connections
del insert_cursor
cursor.close()
connection.close()

## Sample

Justification: By using 90% of the data to train the model, it gets enough information to understand the patterns and connections in the data. Having more data also helps to make the model less biased. This means it gives more accurate results. Keeping 10% of the data aside helps to check if the model's findings apply generally to different sets of data without just memorizing the training data. This way of splitting the data helps to test and confirm how well the model works, especially because real-world data can vary a lot. With more data, we can also spot any unusual things in the data and make sure the model still works well with the rest of the data

In [60]:
# Input feature class
elev_data = os.path.join(db, "elevation_points")

# Get count of features in the input dataset
count = int(arcpy.GetCount_management(elev_data).getOutput(0))

# Number to sample ~10%
sample_size = int(count / 10)

# Generate random indices for selecting points
randoms = random.sample(range(count), sample_size)

# Create feature classes for training and testing samples
training_fc = os.path.join(db, "Training_Elev")
testing_fc = os.path.join(db, "Testing_Elev")

# Check if feature classes exist and delete if they do
for fc in [training_fc, testing_fc]:
    if arcpy.Exists(fc):
        arcpy.Delete_management(fc)

# Create empty feature classes to store selected points
for fc in [training_fc, testing_fc]:
    arcpy.CreateFeatureclass_management(
        out_path=os.path.dirname(fc),
        out_name=os.path.basename(fc),
        geometry_type="POINT",
        spatial_reference=arcpy.Describe(elev_data).spatialReference
    )

# Get fields from the temperature data and add them to the feature classes
fields = [field.name for field in arcpy.ListFields(elev_data) if field.type not in ["OID", "Geometry"]]
for field_name in fields:
    for fc in [training_fc, testing_fc]:
        arcpy.AddField_management(fc, field_name, arcpy.ListFields(elev_data, field_name)[0].type)

# Insert selected points into the appropriate feature classes
with arcpy.da.SearchCursor(elev_data, ["SHAPE@"] + fields) as search_cursor:
    with arcpy.da.Editor(db) as editor:
        with arcpy.da.InsertCursor(training_fc, ["SHAPE@"] + fields) as training_cursor, \
                arcpy.da.InsertCursor(testing_fc, ["SHAPE@"] + fields) as testing_cursor:
            for index, row in enumerate(search_cursor):
                if index in randoms:
                    testing_cursor.insertRow(row)
                else:
                    training_cursor.insertRow(row)

### Leave One Out

In [61]:
# Uses the training data to find best method
arcpy.ga.ExploratoryInterpolation(
    in_features="Training_Elev",
    value_field="grid_code",
    out_cv_table= os.path.join(db, "Elev_Explore_Interpolation"),
    out_geostat_layer=None,
    interp_methods="KERNEL_INTERPOLATION;IDW;UNIVERSAL_KRIGING",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\5572-Lab3\\Lab3.gdb\\Elev_Explore_Interpolation'>

#### Interpolation


Kriging: "An interpolation technique in which the surrounding measured values are weighted to derive a predicted value for an unmeasured location. Weights are based on the distance between the measured points, the prediction locations, and the overall spatial arrangement among the measured points (ESRI, 2024)."

Kernel Interpolation With Barriers: "A moving window predictor that uses the shortest distance between points so that points on either side of the line barriers are connected.Kernel Interpolation is a variant of a first-order Local Polynomial Interpolation in which instability in the calculations is prevented using a method similar to the one used in the ridge regression to estimate the regression coefficients (ESRI, 2024)."

IDW: "(IDW) interpolation determines cell values using a linearly weighted combination of a set of sample points. The weight is a function of inverse distance. The surface being interpolated should be that of a locationally dependent variable (ESRI, 2024)."

In [64]:
#Kriging
elev_kriging = arcpy.sa.Kriging(
    in_point_features="Training_Elev",
    z_field="grid_code",
    kriging_model="Spherical # # # #",
    cell_size=9000,
    search_radius="VARIABLE 12",
    out_variance_prediction_raster=None
)
elev_kriging.save(os.path.join(db, "elev_kriging"))

In [65]:
#IDW
elev_idw = arcpy.sa.Idw(
    in_point_features="Training_Elev",
        z_field="grid_code",
        cell_size=9000,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
elev_idw.save(os.path.join(db, "elev_idw"))

In [67]:
#Kernel
arcpy.ga.KernelInterpolationWithBarriers(
    in_features="Training_Elev",
    z_field="grid_code",
    out_ga_layer="None",
    out_raster=os.path.join(db, "Elev_Kernel"),
    cell_size=9000,
    in_barrier_features=None,
    kernel_function="POLYNOMIAL5",
    bandwidth=None,
    power=1,
    ridge=50,
    output_type="PREDICTION"
)

<Result 'None'>

### Save Interpolated Maps to Geodatabase

In [70]:
#Convert Kriging to point
arcpy.conversion.RasterToPoint(
    in_raster="elev_kriging",
    out_point_features= os.path.join(db, "elev_kriging_point"),
    raster_field="Value"
)
#Save Kriging points to geodatabase
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="elev_kriging_point",
    Output_Geodatabase= os.path.join(arcpy.env.workspace,"Lab3.sde")
)
#Convert IDW to point
arcpy.conversion.RasterToPoint(
    in_raster="elev_idw",
    out_point_features= os.path.join(db,"elev_idw_point"),
    raster_field="Value"
)
#Save IDW points to geodatabase
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="elev_idw_point",
    Output_Geodatabase= os.path.join(arcpy.env.workspace,"Lab3.sde")
)

#Convert Kernel to point
arcpy.conversion.RasterToPoint(
    in_raster="elev_kernel",
    out_point_features= os.path.join(db, "elev_kernel_point"),
    raster_field="Value"
)
#Save Kernel points to geodatabase
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="elev_kernel_point",
    Output_Geodatabase= os.path.join(arcpy.env.workspace,"Lab3.sde")
)
#Push exploratory table to geodatabase
arcpy.conversion.TableToGeodatabase(
    Input_Table=os.path.join(db, "Elev_Explore_Interpolation"),
    Output_Geodatabase= os.path.join(arcpy.env.workspace,"Lab3.sde")
)

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\5572-Lab3\\Lab3.sde'>

### Accuracy Assessment

In [71]:
#Stores difference from interpolation and truth. Kriging was best
elev_point = os.path.join(db, "Testing_Elev")
input_raster = os.path.join(db, "elev_kriging")  
output = os.path.join(db, "Elev_Accur")  

# Copy testing data
arcpy.CopyFeatures_management(elev_point, output)

# Extract raster values to points
extracted_field = "VALUE"  
arcpy.sa.ExtractValuesToPoints(
    in_point_features=elev_point,
    in_raster=input_raster,
    out_point_features= output,
    interpolate_values="NONE",
    add_attributes="VALUE_ONLY"
)

#Calculate a  new field to find the difference between interpolation
# and the actual ground truth
point_value = "grid_code"  
difference = "Difference"  
extracted_field_name = "RASTERVALU"
arcpy.management.CalculateField(
    in_table="Elev_Accur",
    field=difference,
    expression="!grid_code!-!RASTERVALU!",
    expression_type="PYTHON3",
    code_block="",
    field_type="LONG",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

#Save to geodatabase
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="Elev_Accur",
    Output_Geodatabase= os.path.join(arcpy.env.workspace, "Lab3.sde")
)  

<Result '\\\\Mac\\Home\\Documents\\ArcGIS\\Projects\\5572-Lab3\\Lab3.sde'>